Packages laden und Plot-Layout definieren 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
import locale
import matplotlib as mpl

# Setze deutsches Locale (ggf. 'de_DE.UTF-8' auf deinem System verfügbar machen)
locale.setlocale(locale.LC_ALL, 'de_DE.utf8')

# Erlaube den Formatierern, das Locale zu nutzen
mpl.rcParams['axes.formatter.use_locale'] = True

layout_design ={
    # Schriftart und LaTeX
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Latin Modern Sans"],

    # Lade lmodern und sansmath, aktiviere serifenlose Mathematik
    "text.latex.preamble": r"""
        \usepackage{lmodern}
        \usepackage{sansmath}
        \sansmath
        \renewcommand{\familydefault}{\sfdefault}
        \usepackage{icomma}
    """,
    
    "font.size": 13,              # Standard-Schriftgröße für Paper
    "axes.labelsize": 13,          # Achsentitelgröße
    "axes.titlesize": 13,          # Plottitelgröße
    "legend.fontsize": 11,        # Legendentext
    "xtick.labelsize": 11,          # Tick-Label-Größe X
    "ytick.labelsize": 11,          # Tick-Label-Größe Y

    # Plotgröße und Auflösung
    "figure.figsize": (4, 6),      # etwa 15x10 cm für LaTeX-Spaltenbreite
    "figure.dpi": 300,

    # Ränder
    "savefig.bbox": "tight",
    
    # Linien und Gitter
    "axes.grid": True,             # Gitter standardmäßig an
    "grid.color": "gray",
    "grid.linestyle": "--",
    "grid.linewidth": 0.5,
    "grid.alpha": 0.5,            # Transparenz des Gitters
    
    # Achsenlinien
    "axes.edgecolor": "black",
    "axes.linewidth": 0.8,
    
    # Ticks
    "xtick.direction": "in",       # Ticks nach innen
    "ytick.direction": "in",
    "xtick.major.size": 5,         # Ticklänge
    "ytick.major.size": 5,
    "xtick.minor.size": 2.5,       # Kleine Ticks
    "ytick.minor.size": 2.5,
    "lines.markersize": 2,         
}

rcParams.update(layout_design)

Daten laden und Plots erstellen

In [2]:
# Daten als mehrzeiligen String einfügen
df_1 = pd.read_csv('../data/Erster_Spritzdurchgang_Sollprofil.csv', sep=';')
df_2 = pd.read_csv('../data/Zweiter_Spritzdurchgang_Sollprofil.csv', sep=';')
df_3 = pd.read_csv('../data/Dritter_Spritzdurchgang_Sollprofil.csv', sep=';')
df_4 = pd.read_csv('../data/Erster_Zweiter_Spritzdurchgang_Profile.csv', sep=';')
df_5 = pd.read_csv('../data/Erster_Dritter_Spritzdurchgang_Profile.csv', sep=';')


df_dict = {
    'Erster_Spritzdurchgang': df_1,
    'Zweiter_Spritzdurchgang': df_2,
    'Dritter_Spritzdurchgang': df_3,
    'Erster_Zweiter_Spritzdurchgang': df_4,
    'Erster_Dritter_Spritzdurchgang': df_5,
}

df_soll = df_dict['Erster_Spritzdurchgang']
x_soll_vergleich = df_soll['x_soll']
y_soll_vergleich = df_soll['y_soll']

df_soll_1 = df_dict['Dritter_Spritzdurchgang']
x_soll_vergleich_1 = df_soll_1['x_soll']
y_soll_vergleich_1 = df_soll_1['y_soll']

for name, df in df_dict.items():
    # Extrahieren der Spalten
    x_soll = df['x_soll']
    y_soll = df['y_soll']
    x_ist = df['x_ist']
    y_ist = df['y_ist']
    delta = df['delta']

    if name == 'Erster_Zweiter_Spritzdurchgang':
        plt.plot(x_soll, y_soll, label='1. Durchgang', marker='o')
        plt.plot(x_ist, y_ist, label='Vergleichsprofil', marker='o')
        plt.plot(x_soll_vergleich, y_soll_vergleich, label='Sollprofil', color='grey', linestyle='--')
        plt.xlim(min(x_soll_vergleich) - 0.2, max(x_soll_vergleich) + 0.2)
    elif name == 'Erster_Dritter_Spritzdurchgang':
        plt.plot(x_soll, y_soll, label='2. Durchgang', marker='o')
        plt.plot(x_ist, y_ist, label='Vergleichsprofil', marker='o')
        plt.plot(x_soll_vergleich_1, y_soll_vergleich_1, label='Sollprofil', color='grey', linestyle='--')
        plt.xlim(min(x_soll_vergleich_1) - 0.2, max(x_soll_vergleich_1) + 0.2)        
    else:
        plt.plot(x_soll, y_soll, label='Sollprofil', marker='o')
        plt.plot(x_ist, y_ist, label='Istprofil', marker='o')
        plt.xlim(min(x_soll) - 0.1, max(x_soll) + 0.2)

    # Fläche zwischen Soll- und Ist-Kurve segmentweise füllen
    for i in range(len(df) - 1):
        # Eckpunkte für das Polygon
        poly_x = [x_soll[i], x_soll[i+1], x_ist[i+1], x_ist[i]]
        poly_y = [y_soll[i], y_soll[i+1], y_ist[i+1], y_ist[i]]
        # Farbe basierend auf delta am ersten Punkt
        color = 'green' if delta[i] >= 0 else 'red'
        plt.fill(poly_x, poly_y, color=color, alpha=0.3)

    # Abstände rechts der Mittelpunkte annotieren
    for i in range(len(df)):
        mid_x = (x_soll[i] + x_ist[i]) / 2
        mid_y = (y_soll[i] + y_ist[i]) / 2
        label = rf"${delta[i]:.0f}\,\mathrm{{mm}}$"
        if delta[i] >= 0:
            plt.text(x_ist[i] + 0.02, y_ist[i], label, fontsize=8, ha='left', va='center')
        else:
            plt.text(x_soll[i] + 0.02, y_soll[i], label, fontsize=8, ha='left', va='center')

    plt.xlabel(r'\textit{x} in m')
    plt.ylabel(r'\textit{y} in m')
    plt.legend()
    plt.savefig(f'../results/Diagramme/Nassversuche/{name}_Sollprofil.pdf')
    plt.close()
